In [1]:
from DataLoader import MyOwnDataloader
from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchvision

import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

In [2]:
torch.cuda.empty_cache() 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total/1000000}')
print(f'free     : {info.free/1000000}')
print(f'used     : {info.used/1000000}')

total    : 12636.061696
free     : 11491.868672
used     : 1144.193024


In [5]:

dataDir='/media/gamedisk/COCO_dataset/'
val='val2017'
train = 'train2017'

val_annFile='{}/annotations/instances_{}.json'.format(dataDir,val)
train_annFile='{}/annotations/instances_{}.json'.format(dataDir,train) 
# Batch size
batch_size = 8



classes = {
    "bird": 1,
    "cat": 2,
    "dog": 3,
    "horse": 4,
    "sheep": 5,
    "cow": 6,
    "elephant": 7,
    "bear": 8,
    "zebra": 9,
    "giraffe": 10
}


coco = COCO(val_annFile)
val_loader = MyOwnDataloader(dataDir = dataDir, dataType = val,
                     annFile = val_annFile, classes = classes, train_batch_size=batch_size)
valid_dl = val_loader.concat_datasets()


coco = COCO(train_annFile)
train_loader = MyOwnDataloader(dataDir = dataDir, dataType = train,
                     annFile = train_annFile, classes = classes, train_batch_size=batch_size)
train_dl = val_loader.concat_datasets()



loading annotations into memory...
Done (t=0.39s)
creating index...
index created!
loading annotations into memory...
Done (t=0.35s)
creating index...
index created!
loading annotations into memory...
Done (t=8.80s)
creating index...
index created!
loading annotations into memory...
Done (t=9.36s)
creating index...
index created!


In [6]:
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5
num_steps = 50

In [7]:
class Network(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, timesteps: int):
        super(Network, self).__init__()
        self.timesteps = timesteps
        # CNNs for rgb images
        self.conv1 = nn.Conv2d(in_channels, 32,
                        kernel_size=7,
                        padding=3,
                        # no bias because it is not bio-plausible (and hard to impl in neuromorphic hardware)
                        bias=True,
                        dilation=1,
                        stride=2)
        self.lif1 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.conv2 = nn.Conv2d(32, 64,
                               kernel_size=3,
                               padding=1,
                               # no bias because it is not bio-plausible (and hard to impl in neuromorphic hardware)
                               bias=True,
                               stride=2)
        self.lif2 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.conv3 = nn.Conv2d(64, 128,
                        kernel_size=3,
                        padding=1,
                        bias=True,
                        stride=2)
        self.lif3 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.conv4 = nn.Conv2d(128, 256,
                        kernel_size=3,
                        padding=1,
                        # no bias because it is not bio-plausible (and hard to impl in neuromorphic hardware)
                        bias=True,
                        stride=2)
        self.lif4 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.conv5 = nn.Conv2d(256, 512,
                    kernel_size=3,
                    padding=1,
                    bias=True,
                    stride=2)
        self.lif5 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))


        self.flat = nn.Flatten()
        self.dropout = nn.Dropout()
        self.fc = nn.Linear(512, out_channels, bias=True)
        self.fc_spike = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(
            slope=25), init_hidden=False, output=True)


        # # # Connecting CNN outputs with Fully Connected layers for classification

        # self.class_fc1 = nn.Linear(in_features=192*3*3, out_features=240)
        # self.class_fc2 = nn.Linear(in_features=240, out_features=120)
        # self.class_out = nn.Linear(in_features=120, out_features=10)

        self.fc_spike = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(
            slope=25), init_hidden=False, output=True)

        # Connecting CNN outputs with Fully Connected layers for bounding box
        # self.box_fc1 = nn.Linear(in_features=192*3*3, out_features=240)
        # self.box_fc2 = nn.Linear(in_features=240, out_features=120)
        # self.box_out = nn.Linear(in_features=120, out_features=4)


    def forward(self, inputs):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        mem4 = self.lif4.init_leaky()
        mem5 = self.lif5.init_leaky()

        mem_fc_spike = self.fc_spike.init_leaky()

        # Record the final layer
        spk5_rec = []
        mem5_rec = []

        for k in range(self.timesteps):
            t = inputs[k, :, :, :]
            # t = inputs
            t = self.conv1(t)
            t = F.max_pool2d(t, kernel_size=2)
            t, mem1 = self.lif1(t, mem1)

            t = self.conv2(t)
            t = F.max_pool2d(t, kernel_size=2,)
            t, mem2 = self.lif1(t, mem2)


            t = self.conv3(t)
            t = F.max_pool2d(t, kernel_size=2)
            t, mem3 = self.lif1(t, mem3)


            t = self.conv4(t)
            t = F.max_pool2d(t, kernel_size=2)
            t, mem4 = self.lif1(t, mem4)


            t = self.conv5(t)
            # t, mem5 = self.lif1(t, mem5)
            t = F.relu(t)
            t = F.avg_pool2d(t, 1)

            # t = self.avg_pool(t)
            t = torch.flatten(t,start_dim=1)


            # classifier
            # class_t = self.class_fc1(t)
            # class_t = F.relu(class_t)

            # class_t = self.class_fc2(class_t)
            # class_t = F.relu(class_t)


            # class_t, mem_fc_spike = self.fc_spike(class_t, mem_fc_spike)

            class_t = self.flat(t)
            class_t = self.dropout(class_t)
            class_t = class_t.T
            class_t = self.fc(class_t)
            class_t, mem_fc_spike = self.fc_spike(class_t, mem_fc_spike)


            # box_t = self.box_fc1(t)
            # box_t = F.relu(box_t)

            # box_t = self.box_fc2(box_t)
            # box_t = F.relu(box_t)

            # box_t = self.box_out(box_t)
            # box_t = F.sigmoid(box_t)

        return class_t

In [8]:
def get_num_correct(preds, labels):
    return torch.round(preds).argmax(dim=1).eq(labels).sum().item()

In [9]:
def train(model):
    # Defining the optimizer
    optimizer = optim.SGD(model.parameters(),lr = 0.1)
    num_of_epochs = 10
    epochs = []
    losses = []
    # Creating a directory for storing models

    for epoch in range(num_of_epochs):
        tot_loss = 0
        tot_correct = 0
        train_start = time.time()
        model.train()


        for batch, (images, annotations) in tqdm(enumerate(train_dl)):

        	# Converting data from cpu to GPU if available to improve speed
            imgs = list(img.to(device) for img in images)
            neural_images = torch.stack(imgs)
            x = neural_images.to(device)
            annotations = [{k: v for k, v in t.items()} for t in annotations]
            y = annotations[0]['labels'].to(device)
            z = annotations[0]['boxes'].to(device)
            # Sets the gradients of all optimized tensors to zero
            optimizer.zero_grad()
            [y_pred,z_pred]= model(x)
            # Compute loss (here CrossEntropyLoss)
            class_loss = F.cross_entropy(y_pred, y)
            box_loss = F.mse_loss(z_pred, z)
            (box_loss + class_loss).backward()
            # class_loss.backward()
            optimizer.step()
            print("Train batch:", batch+1, " epoch: ", epoch, " ",
                  (time.time()-train_start)/60, end='\r')

        model.eval()
        for batch, (images, annotations) in tqdm(enumerate(valid_dl)):
        	# Converting data from cpu to GPU if available to improve speed	
            imgs = list(img.to(device) for img in images)
            neural_images = torch.stack(imgs)
            x = neural_images.to(device)
            annotations = [{k: v for k, v in t.items()} for t in annotations]
            y = annotations[0]['labels'].to(device)
            z = annotations[0]['boxes'].to(device)
            # Sets the gradients of all optimized tensors to zero
            optimizer.zero_grad()
            with torch.no_grad():
                [y_pred,z_pred]= model(x)
                
                # Compute loss (here CrossEntropyLoss)
                class_loss = F.cross_entropy(y_pred, y)
                box_loss = F.mse_loss(z_pred, z)
                # Compute loss (here CrossEntropyLoss)

            tot_loss += (class_loss.item() + box_loss.item())
            tot_correct += get_num_correct(y_pred, y)
            print("Test batch:", batch+1, " epoch: ", epoch, " ",
                  (time.time()-train_start)/60, end='\r')
        epochs.append(epoch)
        losses.append(tot_loss)
        print("Epoch", epoch, "Accuracy", (tot_correct)/2.4, "loss:",
              tot_loss, " time: ", (time.time()-train_start)/60, " mins")
        torch.save(model.state_dict(), "model_ep"+str(epoch+1)+".pth")

In [10]:
model = Network(3, 10, 8)
model = model.to(device)
model

Network(
  (conv1): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (lif1): Leaky()
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (lif2): Leaky()
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (lif3): Leaky()
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (lif4): Leaky()
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (lif5): Leaky()
  (avg_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flat): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=10, bias=True)
  (fc_spike): Leaky()
)

In [11]:
train(model)

0it [00:00, ?it/s]


RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small